데이터 형태 예제
- CPU:  Intel(R) Xeon(R) Gold 6326 CPU @ 2.90GHz 64 core
- MEM:  DIMM DDR4 Synchronous Registered (Buffered) 3200 MHz (0.3 ns) 32GiB, DIMM DDR4 Synchronous Registered (Buffered) 3200 MHz (0.3 ns) 32GiB,DIMM DDR4 Synchronous Registered (Buffered) 3200 MHz (0.3 ns) 32GiB
- DISK:  SMC3108 SSD 893.8G, SMC3100 SSD 500G
- NETWORK: Intel Corporation Ethernet Controller X710 for 10GBASE-T (rev 02)

In [1]:
import subprocess
import re

from cpuinfo import get_cpu_info

In [2]:
# CPU
cpu_info = get_cpu_info()
cpu = f"{cpu_info['brand_raw']} {cpu_info['count']} core"
cpu

'Intel(R) Xeon(R) Platinum 8259CL CPU @ 2.50GHz 4 core'

In [3]:
# Memory
output = subprocess.check_output(["sudo", "lshw", "-c", "memory"])
decoded_output = output.decode()

lines = [i for i in decoded_output.split("*-") if i[:4] == "bank"]
memory = []

for line in lines:
    pattern_description = re.compile(r"description: (.+)\n")
    pattern_size = re.compile(r"size: (.+)\n")
    
    description = pattern_description.search(line).group(1)
    if "empty" in description:
        continue
    size = pattern_size.search(line).group(1)

    memory.append(f"{description} {size}")

memory

['DIMM DDR4 Static column Pseudo-static Synchronous Window DRAM 2933 MHz (0.3 ns) 16GiB']

In [4]:
# Disk
output = subprocess.check_output(["sudo", "lshw", "-c", "disk"])
decoded_output = output.decode()

lines = [i for i in decoded_output.split("*-") if i.strip()]
disk = []

for line in lines:
    pattern_description = re.compile(r"description: (.+)\n")
    pattern_product = re.compile(r"product: (.+)\n")
    pattern_capacity = re.compile(r"capacity: (.+)\n")

    description = pattern_description.search(line).group(1)
    product = pattern_product.search(line).group(1)
    capacity = pattern_capacity.search(line).group(1)
    
    disk.append(f"[{description}] {product} {capacity}")

disk

[]

In [5]:
# Network
output = subprocess.check_output(["sudo", "lshw", "-c", "network"])
decoded_output = output.decode()

lines = [i for i in decoded_output.split("*-") if i.strip()]
network = []

for line in lines:
    pattern_product = re.compile(r"product: (.+)\n")
    
    searched = pattern_product.search(line)
    if not searched:
        continue
    product = searched.group(1)
    
    if not product in network:
        network.append(f"{product}")

network

['Elastic Network Adapter (ENA)']

In [2]:
from kubernetes import client, config
from app.get_info import *
from datetime import datetime

In [3]:
# Load kubernetes config
cfg = config.load_kube_config()

# Create kubernetes client
k8s_client = client.ApiClient(cfg)

# Create kubernetes v1
v1 = client.CoreV1Api(k8s_client)

node_name = "ip-172-31-57-102.ap-northeast-2.compute.internal"

In [31]:
hardware_info = {
    "updated_datetime": datetime.now().isoformat(),
    "cpu": cpu_info(),
    "memory": memory_info(),
    "disk": disk_info(),
    "network": network_info(),
}

In [32]:
# Get node annotations
node = v1.read_node(name=node_name)
hardware_info = str(hardware_info).replace("'", '"')
node.metadata.annotations["hardware-info"] = hardware_info

In [34]:
# Patch node annotations
v1.patch_node(name=node_name, body=node)

{'api_version': 'v1',
 'kind': 'Node',
 'metadata': {'annotations': {'flannel.alpha.coreos.com/backend-data': '{"VNI":1,"VtepMAC":"82:d2:60:d4:e9:1d"}',
                              'flannel.alpha.coreos.com/backend-type': 'vxlan',
                              'flannel.alpha.coreos.com/kube-subnet-manager': 'true',
                              'flannel.alpha.coreos.com/public-ip': '172.31.57.102',
                              'hardware-info': '{"updated_datetime": '
                                               '"2023-06-25T13:41:05.243288", '
                                               '"cpu": "Intel(R) Xeon(R) '
                                               'Platinum 8259CL CPU @ 2.50GHz '
                                               '4 core", "memory": [], "disk": '
                                               '[], "network": ["Elastic '
                                               'Network Adapter (ENA)"]}',
                              'kubeadm.alpha.kubernetes.io/

In [13]:
import json

node = v1.read_node(name=node_name)

hw_info = json.loads(node.metadata.annotations["hardware-info"])

In [14]:
hw_info

{'updated_datetime': '2023-06-25T13:58:41.381626',
 'cpu': 'Intel(R) Xeon(R) Platinum 8259CL CPU @ 2.50GHz 4 core',
 'memory': ['DIMM DDR4 Static column Pseudo-static Synchronous Window DRAM 2933 MHz (0.3 ns) 16GiB'],
 'disk': ['[NVMe disk] 50GiB (53GB)'],
 'network': ['Elastic Network Adapter (ENA)']}